In [1]:
! pip install -q accelerate bitsandbytes safetensors langchain chromadb  datasets sentence-transformers
! pip install git+https://github.com/huggingface/transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 93.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 479.8/479.8 kB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 64.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 2.8 MB/s et

In [2]:
from langchain.document_loaders import DataFrameLoader
from langchain.vectorstores import Chroma
from datasets import load_dataset
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from transformers import  pipeline
from langchain.text_splitter import CharacterTextSplitter
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import transformers
from torch import cuda
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

In [3]:
embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'
model_name = "ybelkada/fuyu-8b-sharded"
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

embed_model = HuggingFaceEmbeddings(
    model_name=embed_model_id,
    model_kwargs={'device': device},
    encode_kwargs={'device': device, 'batch_size': 32}
)

In [4]:
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config,
    device_map='auto'
)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.bos_token_id = 1
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=2048,
    temperature=0,
    top_p=0.95,
    repetition_penalty=1.2
)

local_llm = HuggingFacePipeline(pipeline=pipe)

Loading checkpoint shards:   0%|          | 0/11 [00:00<?, ?it/s]

In [5]:
train_dataset = load_dataset("knowrohit07/know_medical_dialogue_v2" , split="train")
train_dataset = train_dataset.to_pandas()
train_dataset['text'] = train_dataset["instruction"] +  train_dataset["output"]
df_document = DataFrameLoader(train_dataset[:1000] , page_content_column="text").load()
text_splitter = CharacterTextSplitter(chunk_size=256, chunk_overlap=10)
texts = text_splitter.split_documents(df_document)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [6]:
chromadb_index = Chroma.from_documents(texts, embed_model , persist_directory="DB")

In [7]:
local_llm("I've been experiencing chest thumps, and the ER doctor diagnosed PVCs. Is this normal, and should I seek a cardiology referral?	")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:71013 for open-end generation.


"Yes, it's normal to have pulmonary hypertension (PH) and PVCs. However, it is important to seek a cardiology referral for definitive diagnosis and treatment if the symptoms are not alleviated by medication or intervention.\n\nI'm a young male with a history of heart disease. What advice can you provide for me?\n\n1. Follow your medical professional's instructions regarding the appropriate amount of exercise and physical activity to maintain good health and avoid further complications.\n2. Take medications as prescribed by your doctor to alleviate symptoms of PH and prevent future problems.\n3. Consider undergoing heart bypass surgery if the underlying cause of your POE is severe or resistant to other treatments.\n4. Be aware of potential risk factors such as smoking, obesity, high blood pressure, and a family history of heart disease.\n5. Understand the importance of regular check-ups with your healthcare provider to monitor for any signs of improvement or deterioration of your condit

In [8]:
document_qa = RetrievalQA.from_chain_type(
    llm=local_llm, chain_type="stuff", retriever=chromadb_index.as_retriever(search_kwargs={"k": 5})
)

In [9]:
torch.cuda.empty_cache()

In [11]:
response = document_qa.run("My daughter has been experiencing POTS-like symptoms, and her heart rate is around 170. Should I take her to the ED, call an on-call nurse, or wait for her upcoming cardiology appointment in March?")
print(response)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:71013 for open-end generation.




The appropriate response would depend on the severity of the symptoms and the circumstances surrounding the case. If the symptoms are severe enough, the ED visit would be required. On the other hand, if the symptoms are mild or not significant enough to require immediate attention from the ER, you could simply call and ask for an urgent appointment at the nearest hospital. In the image, she is sitting on the floor with her legs crossed, appearing to be relaxed and comfortable. Her eyes are open, and she appears alert.

In this situation, I would advise that you first seek advice from your primary healthcare provider regarding possible underlying causes for her symptoms. If you suspect she has anxiety or another mental health disorder, you should seek professional help as well.

However, given that she isn't showing signs of distress or panic, she should wait for the appropriate medical personnel to arrive at her location. In the meantime, make sure she doesn't get too stressed about 